In [4]:
from task2vec import Task2Vec
from models import get_model
import datasets
import task_similarity
import argparse
import torch
import torch.nn.functional as F
import torchvision.transforms as tt
from tqdm import tqdm
from easydict import EasyDict as edict

In [35]:
args = edict()
args.batch_size = 135
args.n_views = 2
args.device = 'cpu'
args.temperature = 3

In [36]:
saved = torch.load('all.pt')
seed = 123

In [59]:
Z = [z.hessian for z in saved]
features = torch.cat([torch.tensor(Z),torch.tensor(Z)], axis = 0)

In [60]:
labels = torch.cat([torch.arange(args.batch_size) for i in range(args.n_views)], dim=0)
labels = (labels.unsqueeze(0) == labels.unsqueeze(1)).float()

features = F.normalize(features, dim=1)

similarity_matrix = torch.matmul(features, features.T)

In [61]:
mask = torch.eye(labels.shape[0], dtype=torch.bool)
labels = labels[~mask].view(labels.shape[0], -1)

In [62]:
similarity_matrix = similarity_matrix[~mask].view(similarity_matrix.shape[0], -1)

In [63]:


# assert similarity_matrix.shape == (
#     self.args.n_views * self.args.batch_size, self.args.n_views * self.args.batch_size)
# assert similarity_matrix.shape == labels.shape

# discard the main diagonal from both: labels and similarities matrix

# assert similarity_matrix.shape == labels.shape

# select and combine multiple positives
positives = similarity_matrix[labels.bool()].view(labels.shape[0], -1)

# select only the negatives the negatives
negatives = similarity_matrix[~labels.bool()].view(similarity_matrix.shape[0], -1)

logits = torch.cat([positives, negatives], dim=1)
labels = torch.zeros(logits.shape[0], dtype=torch.long)

logits = logits / args.temperature

In [64]:
negatives.shape

torch.Size([270, 268])

In [75]:
logits

tensor([[0.3333, 0.3088, 0.2974,  ..., 0.3087, 0.3167, 0.3159],
        [0.3333, 0.3088, 0.3003,  ..., 0.3209, 0.3149, 0.3139],
        [0.3333, 0.2974, 0.3003,  ..., 0.3085, 0.3000, 0.2990],
        ...,
        [0.3333, 0.3087, 0.3209,  ..., 0.3097, 0.3119, 0.3100],
        [0.3333, 0.3167, 0.3149,  ..., 0.3042, 0.3119, 0.3193],
        [0.3333, 0.3159, 0.3139,  ..., 0.3033, 0.3100, 0.3193]])

In [68]:
logits.shape

torch.Size([270, 269])